In [214]:
import random
import time
import warnings
import argparse
import shutil
import os.path as osp
from torchsummary import summary
import os
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.optim import SGD
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
import torch.nn.functional as F
import cv2
import numpy as np
import custom_utils
from matplotlib import pyplot as plt
from custom_utils import plot_graph
from torch.utils.data import Dataset, DataLoader
from tllib.alignment.dan import MultipleKernelMaximumMeanDiscrepancy, ImageClassifier
from tllib.modules.kernels import GaussianKernel
from tllib.utils.data import ForeverDataIterator
from tllib.utils.metric import accuracy
from tllib.utils.meter import AverageMeter, ProgressMeter
from tllib.utils.logger import CompleteLogger
from tllib.utils.analysis import collect_feature, tsne, a_distance
import pandas as pd
import numpy
import gc
torch.set_printoptions(profile="full")
gc.collect()

21

In [215]:
def remapping(df, map):
    df_copy = df.copy()
    df_copy['Label'] = df_copy['Label'].replace(map)
    return df_copy

In [216]:
# Set data path
Train_path = '/home/bkcs/HDD/FL/Data_Processing/Data/Concatenated/Full_data/Concatenated_train.feather'
Test_path = '/home/bkcs/HDD/FL/Data_Processing/Data/Concatenated/Full_data/Concatenated_test.feather'


Train_data = pd.read_feather(Train_path)
Test_data = pd.read_feather(Test_path)



In [217]:
Train_data.columns

Index(['flow_id', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '1015', '1016', '1017', '1018', '1019', '1020', '1021', '1022', '1023',
       'Label'],
      dtype='object', length=1026)

In [218]:
Train_data['Label'].value_counts()/20

11    4728.0
4     3715.0
0     2850.0
2     2784.0
1     2042.0
12    2036.0
3     1776.0
17    1512.0
9     1475.0
7     1338.0
5     1132.0
10    1119.0
14     954.0
16     726.0
6      668.0
13     402.0
15     236.0
8      157.0
Name: Label, dtype: float64

In [219]:
mapping_dict = {'FileTransfer': 0, 'GoogleHangout_Chat': 1, 'Music': 2, 'VoIP': 3, 'Youtube': 4, 'alibaba': 5, 'amazon': 6, 'docs': 7, 'drive': 8, 'ebay': 9, 'facebook': 10, 'photo': 11, 'shopee': 12, 'thegioididong': 13, 'tiki': 14, 'tiktok': 15, 'youtube': 16, 'youtubelive': 17}
reverse_mapping_dict = {v: k for k, v in mapping_dict.items()}
Train_data['Label'].replace(reverse_mapping_dict,inplace=True)
Test_data['Label'].replace(reverse_mapping_dict,inplace=True)


In [220]:
Train_data['Label'].value_counts()/20

photo                 4728.0
Youtube               3715.0
FileTransfer          2850.0
Music                 2784.0
GoogleHangout_Chat    2042.0
shopee                2036.0
VoIP                  1776.0
youtubelive           1512.0
ebay                  1475.0
docs                  1338.0
alibaba               1132.0
facebook              1119.0
tiki                   954.0
youtube                726.0
amazon                 668.0
thegioididong          402.0
tiktok                 236.0
drive                  157.0
Name: Label, dtype: float64

In [221]:
Train_data['flow_id'].value_counts().describe()

count    29650.0
mean        20.0
std          0.0
min         20.0
25%         20.0
50%         20.0
75%         20.0
max         20.0
Name: flow_id, dtype: float64

In [222]:
# ['Ecommerce', 'Video', 'FileTransfer', 'Google_service']
# ['Ecommerce', 'Video', 'Google_service']
# original_dict = {0: ['thegioididong','amazon','tiki','alibaba','ebay','shopee'],
#                  1: ['tiktok','youtube','facebook','youtubelive','Youtube'],
#                  2: ['drive','FileTransfer'],
#                  3: ['docs','VoIP','Music','photo','GoogleHangout_Chat']}
original_dict = {0: ['thegioididong','amazon','tiki','alibaba','ebay','shopee'],
                 1: ['tiktok','youtube','facebook','youtubelive','Youtube'],
                 2: ['docs','VoIP','Music','photo','GoogleHangout_Chat']}
label_mapping = {}

for key, values in original_dict.items():
    for value in values:
        label_mapping[value] = key

print(label_mapping)


{'thegioididong': 0, 'amazon': 0, 'tiki': 0, 'alibaba': 0, 'ebay': 0, 'shopee': 0, 'tiktok': 1, 'youtube': 1, 'facebook': 1, 'youtubelive': 1, 'Youtube': 1, 'docs': 2, 'VoIP': 2, 'Music': 2, 'photo': 2, 'GoogleHangout_Chat': 2}


In [223]:
def test_case_split(df,split):
    selected = df['flow_id'].drop_duplicates().sample(frac=1)
    # print(selected.shape)
    total_data_count = selected.shape[0]
    data_per_set = int(np.floor(total_data_count/split))
    DataFrameDict = {}
    for i in range(1, split+1):
        client_name = "client_" + str(i)
        start = data_per_set * (i-1)
        end = data_per_set * i

        print(f"Adding data from {start} to {end} for client : {client_name}")
        DataFrameDict[client_name] = df[df['flow_id'].isin(selected[start:end])]
    return DataFrameDict

In [224]:
source_labels = ['Youtube', 'GoogleHangout_Chat','shopee','tiki','thegioididong']
target_labels = ['ebay','alibaba','amazon','VoIP','facebook']

# Group 1
train_source = Train_data.loc[Train_data['Label'].isin(source_labels)]
val_raw = Test_data.loc[Test_data['Label'].isin(source_labels)]


In [225]:
# Group 2
# Traning
# train_target = Train_data.loc[Train_data['Label'].isin(target_labels)] # old code
train_target_1 = Train_data.loc[Train_data['Label'].isin(['ebay'])]
train_target_2 = Train_data.loc[Train_data['Label'].isin(['alibaba'])]
train_target_3 = Train_data.loc[Train_data['Label'].isin(['amazon'])]

train_VoIP_facebook = Train_data.loc[Train_data['Label'].isin(['VoIP', 'facebook'])]
train_DataFrameDict = test_case_split(train_VoIP_facebook,3)


Adding data from 0 to 965 for client : client_1
Adding data from 965 to 1930 for client : client_2
Adding data from 1930 to 2895 for client : client_3


In [226]:
train_VoIP_facebook.head(10)

,flow_id,0,1,2,3,4,5,6,7,8,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,Label
44600,4080,93,169,121,35,113,96,229,158,250,...,0,0,0,0,0,0,0,0,0,facebook
44601,4080,93,169,121,195,4,90,253,175,20,...,0,0,0,0,0,0,0,0,0,facebook
44602,4080,101,149,3,21,89,135,144,250,129,...,0,0,0,0,0,0,0,0,0,facebook
44603,4080,93,169,121,78,203,129,48,255,198,...,0,0,0,0,0,0,0,0,0,facebook
44604,4080,101,149,3,21,89,135,144,250,132,...,0,0,0,0,0,0,0,0,0,facebook
44605,4080,93,169,121,75,236,231,169,223,36,...,0,0,0,0,0,0,0,0,0,facebook
44606,4080,93,169,121,184,165,86,83,46,175,...,0,0,0,0,0,0,0,0,0,facebook
44607,4080,101,149,3,21,89,135,144,250,231,...,0,0,0,0,0,0,0,0,0,facebook
44608,4080,93,169,121,170,222,93,197,219,180,...,0,0,0,0,0,0,0,0,0,facebook
44609,4080,101,149,3,21,89,135,144,250,206,...,0,0,0,0,0,0,0,0,0,facebook


In [227]:
train_VoIP_facebook.tail(10)

,flow_id,0,1,2,3,4,5,6,7,8,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,Label
552150,212986,144,111,87,2,137,119,99,165,151,...,0,0,0,0,0,0,0,0,0,VoIP
552151,212986,144,111,87,13,137,119,140,229,151,...,0,0,0,0,0,0,0,0,0,VoIP
552152,212986,144,111,87,4,137,119,107,37,151,...,0,0,0,0,0,0,0,0,0,VoIP
552153,212986,144,111,87,15,137,119,148,101,151,...,0,0,0,0,0,0,0,0,0,VoIP
552154,212986,144,111,87,6,137,119,114,165,151,...,0,0,0,0,0,0,0,0,0,VoIP
552155,212986,144,111,87,0,137,119,92,37,151,...,0,0,0,0,0,0,0,0,0,VoIP
552156,212986,144,111,87,17,137,119,155,229,151,...,0,0,0,0,0,0,0,0,0,VoIP
552157,212986,144,111,87,1,137,119,95,229,151,...,0,0,0,0,0,0,0,0,0,VoIP
552158,212986,144,111,87,3,137,119,103,101,151,...,0,0,0,0,0,0,0,0,0,VoIP
552159,212986,144,111,87,16,137,119,152,37,151,...,0,0,0,0,0,0,0,0,0,VoIP


In [228]:
train_DataFrameDict['client_1']

,flow_id,0,1,2,3,4,5,6,7,8,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,Label
44660,4085,99,219,2,214,12,25,77,24,92,...,0,0,0,0,0,0,0,0,0,facebook
44661,4085,99,219,2,214,12,25,77,24,245,...,0,0,0,0,0,0,0,0,0,facebook
44662,4085,176,238,56,5,240,160,150,87,169,...,0,0,0,0,0,0,0,0,0,facebook
44663,4085,99,219,2,214,12,25,77,24,117,...,0,0,0,0,0,0,0,0,0,facebook
44664,4085,176,238,56,90,103,133,123,218,250,...,0,0,0,0,0,0,0,0,0,facebook
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552155,212986,144,111,87,0,137,119,92,37,151,...,0,0,0,0,0,0,0,0,0,VoIP
552156,212986,144,111,87,17,137,119,155,229,151,...,0,0,0,0,0,0,0,0,0,VoIP
552157,212986,144,111,87,1,137,119,95,229,151,...,0,0,0,0,0,0,0,0,0,VoIP
552158,212986,144,111,87,3,137,119,103,101,151,...,0,0,0,0,0,0,0,0,0,VoIP


In [229]:
train_target_1

,flow_id,0,1,2,3,4,5,6,7,8,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,Label
6080,573,23,3,3,6,153,30,236,72,162,...,203,130,214,204,26,226,154,234,251,ebay
6081,573,23,3,3,5,163,91,172,25,225,...,13,253,160,154,170,163,194,151,150,ebay
6082,573,23,3,3,2,199,216,47,23,254,...,0,0,0,0,0,0,0,0,0,ebay
6083,573,23,3,3,5,208,25,231,182,62,...,247,110,114,112,55,232,174,221,169,ebay
6084,573,23,3,3,0,26,97,172,74,132,...,0,0,0,0,0,0,0,0,0,ebay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329655,74621,23,3,3,5,115,201,3,136,118,...,91,190,108,6,78,79,22,16,82,ebay
329656,74621,23,3,3,5,115,126,9,108,158,...,160,248,125,36,106,98,43,117,22,ebay
329657,74621,23,3,3,5,115,104,254,165,35,...,24,56,21,140,138,69,29,3,31,ebay
329658,74621,23,3,3,0,26,246,23,57,5,...,0,0,0,0,0,0,0,0,0,ebay


In [230]:
train_target_1 = pd.concat([train_target_1,train_DataFrameDict['client_1']],axis=0)
train_target_2 = pd.concat([train_target_2,train_DataFrameDict['client_2']],axis=0)
train_target_3 = pd.concat([train_target_3,train_DataFrameDict['client_3']],axis=0)

del Train_data, train_VoIP_facebook, train_DataFrameDict

In [231]:
train_target_1.head(10)

,flow_id,0,1,2,3,4,5,6,7,8,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,Label
6080,573,23,3,3,6,153,30,236,72,162,...,203,130,214,204,26,226,154,234,251,ebay
6081,573,23,3,3,5,163,91,172,25,225,...,13,253,160,154,170,163,194,151,150,ebay
6082,573,23,3,3,2,199,216,47,23,254,...,0,0,0,0,0,0,0,0,0,ebay
6083,573,23,3,3,5,208,25,231,182,62,...,247,110,114,112,55,232,174,221,169,ebay
6084,573,23,3,3,0,26,97,172,74,132,...,0,0,0,0,0,0,0,0,0,ebay
6085,573,23,3,3,2,221,85,45,228,85,...,0,0,0,0,0,0,0,0,0,ebay
6086,573,23,3,3,4,209,183,15,223,225,...,23,4,42,222,242,218,32,230,184,ebay
6087,573,23,3,3,11,81,159,199,94,143,...,131,11,48,254,67,14,143,243,86,ebay
6088,573,23,3,3,5,143,65,171,86,73,...,140,213,201,157,160,239,36,3,220,ebay
6089,573,23,3,3,1,192,62,204,54,22,...,0,0,0,0,0,0,0,0,0,ebay


In [232]:
# Test
# test_raw =  Test_data.loc[Test_data['Label'].isin(target_labels)] # old code
test_target_1 = Test_data.loc[Test_data['Label'].isin(['ebay'])]
test_target_2 = Test_data.loc[Test_data['Label'].isin(['alibaba'])]
test_target_3 = Test_data.loc[Test_data['Label'].isin(['amazon'])]

test_VoIP_facebook = Test_data.loc[Test_data['Label'].isin(['VoIP', 'facebook'])]
test_DataFrameDict = test_case_split(test_VoIP_facebook,3)

test_target_1 = pd.concat([test_target_1,test_DataFrameDict['client_1']],axis=0)
test_target_2 = pd.concat([test_target_2,test_DataFrameDict['client_2']],axis=0)
test_target_3 = pd.concat([test_target_3,test_DataFrameDict['client_3']],axis=0)

del Test_data, test_VoIP_facebook, test_DataFrameDict

Adding data from 0 to 227 for client : client_1
Adding data from 227 to 454 for client : client_2
Adding data from 454 to 681 for client : client_3


In [233]:
train_source = remapping(train_source, label_mapping)
val_raw = remapping(val_raw, label_mapping)

train_target_1 = remapping(train_target_1, label_mapping)
train_target_2 = remapping(train_target_2, label_mapping)
train_target_3 = remapping(train_target_3, label_mapping)

test_target_1 = remapping(test_target_1, label_mapping)
test_target_2 = remapping(test_target_2, label_mapping)
test_target_3 = remapping(test_target_3, label_mapping)

In [234]:
train_source['Label'].value_counts()/20

1    3715.0
0    3392.0
2    2042.0
Name: Label, dtype: float64

In [235]:
val_raw['Label'].value_counts()/20

1    930.0
0    903.0
2    514.0
Name: Label, dtype: float64

In [236]:
train_target_1.head(20)

,flow_id,0,1,2,3,4,5,6,7,8,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,Label
6080,573,23,3,3,6,153,30,236,72,162,...,203,130,214,204,26,226,154,234,251,0
6081,573,23,3,3,5,163,91,172,25,225,...,13,253,160,154,170,163,194,151,150,0
6082,573,23,3,3,2,199,216,47,23,254,...,0,0,0,0,0,0,0,0,0,0
6083,573,23,3,3,5,208,25,231,182,62,...,247,110,114,112,55,232,174,221,169,0
6084,573,23,3,3,0,26,97,172,74,132,...,0,0,0,0,0,0,0,0,0,0
6085,573,23,3,3,2,221,85,45,228,85,...,0,0,0,0,0,0,0,0,0,0
6086,573,23,3,3,4,209,183,15,223,225,...,23,4,42,222,242,218,32,230,184,0
6087,573,23,3,3,11,81,159,199,94,143,...,131,11,48,254,67,14,143,243,86,0
6088,573,23,3,3,5,143,65,171,86,73,...,140,213,201,157,160,239,36,3,220,0
6089,573,23,3,3,1,192,62,204,54,22,...,0,0,0,0,0,0,0,0,0,0


In [237]:
train_target_1['Label'].value_counts()/20

0    1475.0
2     593.0
1     372.0
Name: Label, dtype: float64

In [238]:
train_target_2['Label'].value_counts()/20

0    1132.0
2     584.0
1     381.0
Name: Label, dtype: float64

In [239]:
train_target_3['Label'].value_counts()/20

0    668.0
2    599.0
1    366.0
Name: Label, dtype: float64

In [240]:
test_target_1['Label'].value_counts()/20

0    361.0
2    136.0
1     91.0
Name: Label, dtype: float64

In [241]:
test_target_2['Label'].value_counts()/20

0    269.0
2    138.0
1     89.0
Name: Label, dtype: float64

In [242]:
test_target_3['Label'].value_counts()/20

0    166.0
2    148.0
1     79.0
Name: Label, dtype: float64

In [243]:
train_source.reset_index(drop=True, inplace=True)
val_raw.reset_index(drop=True, inplace=True)

train_target_1.reset_index(drop=True, inplace=True)
train_target_2.reset_index(drop=True, inplace=True)
train_target_3.reset_index(drop=True, inplace=True)

test_target_1.reset_index(drop=True, inplace=True)
test_target_2.reset_index(drop=True, inplace=True)
test_target_3.reset_index(drop=True, inplace=True)

In [244]:
train_source.shape[0]/20


9149.0

In [245]:
val_raw.shape[0]/20

2347.0

In [246]:
train_target_1.shape[0]/20


2440.0

In [247]:
train_target_2.shape[0]/20

2097.0

In [248]:
train_target_3.shape[0]/20

1633.0

In [249]:
test_target_1.shape[0]/20

588.0

In [250]:
test_target_2.shape[0]/20

496.0

In [251]:
test_target_3.shape[0]/20

393.0

In [252]:
# # Plotting the first ten rows of the DataFrame and saving as images
# plt.figure(figsize=(10, 15))

# for i in range(10):
#     plt.subplot(10, 1, i + 1)
#     plt.imshow(test_target_1.iloc[i:i + 1], cmap='viridis', aspect='auto')
#     plt.colorbar()
#     plt.title(f'Row {i+1} Visualization')
#     plt.xlabel('Columns')
#     plt.ylabel('Row')

# plt.tight_layout()
# plt.savefig('first_ten_rows.png')  # Save the visualizations of the first ten rows as an image file
# plt.show()


In [253]:
def get_nbyte(df, num_byte):
    df = pd.concat([df.loc[:, : num_byte], df.iloc[:, -1:]], axis=1)
    return df

In [254]:
data_bytes = ['9', '31', '63', '127', '255', '383', '511', '1023']

for each_byte in data_bytes:
    sub_train_source = get_nbyte(train_source, each_byte)
    sub_val_raw = get_nbyte(val_raw, each_byte)
    
    sub_train_target_1 = get_nbyte(train_target_1, each_byte)
    sub_train_target_2 = get_nbyte(train_target_2, each_byte)
    sub_train_target_3 = get_nbyte(train_target_3, each_byte)
    
    sub_test_target_1 = get_nbyte(test_target_1, each_byte)
    sub_test_target_2 = get_nbyte(test_target_2, each_byte)
    sub_test_target_3 = get_nbyte(test_target_3, each_byte)
    
    each_byte = str(int(each_byte) + 1)
    train_source_name = "train_source_" + each_byte + ".feather"
    val_raw_name = "val_raw_" + each_byte + ".feather"
    
    train_target_1_name = "train_target_1_" + each_byte + ".feather"
    train_target_2_name = "train_target_2_" + each_byte + ".feather"
    train_target_3_name = "train_target_3_" + each_byte + ".feather"
    
    test_target_1_name = "test_target_1_" + each_byte + ".feather"
    test_target_2_name = "test_target_2_" + each_byte + ".feather"
    test_target_3_name = "test_target_3_" + each_byte + ".feather"

    path = "/home/bkcs/HDD/Transfer-Learning-Library/examples/domain_adaptation/image_classification/data/concat/"

    sub_train_source.to_feather(path + train_source_name)
    sub_val_raw.to_feather(path + val_raw_name)
    
    sub_train_target_1.to_feather(path + train_target_1_name)
    sub_train_target_2.to_feather(path + train_target_2_name)
    sub_train_target_3.to_feather(path + train_target_3_name)
    
    sub_test_target_1.to_feather(path + test_target_1_name)
    sub_test_target_2.to_feather(path + test_target_2_name)
    sub_test_target_3.to_feather(path + test_target_3_name)


In [255]:
import datetime
import time

# Start the timer
start_time = time.time()

# Your code for model training, validation, or testing goes here
# ...

# Calculate the elapsed time
elapsed_time = time.time() - start_time

# Convert elapsed time to hours, minutes, seconds, and smaller units of seconds
hours, rem = divmod(elapsed_time, 3600)
minutes, rem = divmod(rem, 60)
seconds, microseconds = divmod(rem, 1)
microseconds = round(microseconds, 3)

# Print the elapsed time
print("Elapsed time: {:0>2}:{:0>2}:{:05.3f}".format(int(hours), int(minutes), seconds + microseconds))


Elapsed time: 00:00:0.000
